# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera:** Ing. Sistemas Computacionales </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 11**: Multi classification (Decision Trees & SVM)

**Fecha**: 10/05/2025

**Nombre del Estudiante**:
- Jose Cota 
- Andres Blanco
- Jose Hurtado
- Jose Mendoza

**Profesor**: Pablo Camarillo Ramirez

In [32]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [33]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Decision-Trees") \
    .master("spark://2815f8f35b28:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

### Preparación de Datos

In [34]:
from team_name.spark_utils import SparkUtils
from pyspark.sql.functions import col, trim

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("Id", "string"), ("SepalLengthCm", "float"), ("SepalWidthCm", "float"),
                                    ("PetalLengthCm", "float"), ("PetalWidthCm", "float"), ("Species", "string")])

# Read data Iris CSV
iris_data = spark.read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/Iris.csv")

# Cast del id string a int y a df
df = iris_data.withColumn("Id", trim(col("Id")).cast("int"))
df.show()

+---+-------------+------------+-------------+------------+------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|     Species|
+---+-------------+------------+-------------+------------+------------+
|  1|          5.1|         3.5|          1.4|         0.2| Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2| Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2| Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2| Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2| Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4| Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3| Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2| Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2| Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1| Iris-setosa|
| 11|          5.4|         3.7|          1.5|     

## String to numeric label


In [35]:
from pyspark.ml.feature import StringIndexer

label_indexer = StringIndexer(inputCol="Species", outputCol="label")
data = label_indexer.fit(df).transform(df)
data.show(15) # Para comprobar el label a valor numerico

+---+-------------+------------+-------------+------------+------------+-----+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|     Species|label|
+---+-------------+------------+-------------+------------+------------+-----+
|  1|          5.1|         3.5|          1.4|         0.2| Iris-setosa|  0.0|
|  2|          4.9|         3.0|          1.4|         0.2| Iris-setosa|  0.0|
|  3|          4.7|         3.2|          1.3|         0.2| Iris-setosa|  0.0|
|  4|          4.6|         3.1|          1.5|         0.2| Iris-setosa|  0.0|
|  5|          5.0|         3.6|          1.4|         0.2| Iris-setosa|  0.0|
|  6|          5.4|         3.9|          1.7|         0.4| Iris-setosa|  0.0|
|  7|          4.6|         3.4|          1.4|         0.3| Iris-setosa|  0.0|
|  8|          5.0|         3.4|          1.5|         0.2| Iris-setosa|  0.0|
|  9|          4.4|         2.9|          1.4|         0.2| Iris-setosa|  0.0|
| 10|          4.9|         3.1|          1.5|      

### Assemble the features into a single vector column

In [36]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features" )
data_with_features = assembler.transform(data).select("label", "features")
data_with_features.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[5.09999990463256...|
|  0.0|[4.90000009536743...|
|  0.0|[4.69999980926513...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.5999999046...|
+-----+--------------------+
only showing top 5 rows



### Split the data into training and test sets 80% training data and 20% testing data

In [37]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [38]:
print("Original Dataset")
data_with_features.show()

# Print train dataset
print("train set")
train_df.show()

Original Dataset
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[5.09999990463256...|
|  0.0|[4.90000009536743...|
|  0.0|[4.69999980926513...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.5999999046...|
|  0.0|[5.40000009536743...|
|  0.0|[4.59999990463256...|
|  0.0|[5.0,3.4000000953...|
|  0.0|[4.40000009536743...|
|  0.0|[4.90000009536743...|
|  0.0|[5.40000009536743...|
|  0.0|[4.80000019073486...|
|  0.0|[4.80000019073486...|
|  0.0|[4.30000019073486...|
|  0.0|[5.80000019073486...|
|  0.0|[5.69999980926513...|
|  0.0|[5.40000009536743...|
|  0.0|[5.09999990463256...|
|  0.0|[5.69999980926513...|
|  0.0|[5.09999990463256...|
+-----+--------------------+
only showing top 20 rows

train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[4.30000019073486...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.40000009536743...|
|  0.0|[4.5,2.2999999523...|
|  0.0|[4.599999904

# Decision Tree Model (Training and Predictions) - Iris

In [39]:
from pyspark.ml.classification import DecisionTreeClassifier

# Initialize and train the Decision Tree model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

# Training
dt_model = dt.fit(train_df)

# Display model summary
print("Decision Tree model summary:{0}".format(dt_model.toDebugString))


# Predict
# Use the trained model to make predictions on the test data
predictions = dt_model.transform(test_df)

# Show predictions
print("PREDICTIONS")
predictions.select("features", "prediction").show(30)

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f0e8794db825, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.449999988079071)
   Predict: 0.0
  Else (feature 2 > 2.449999988079071)
   If (feature 2 <= 4.8500001430511475)
    If (feature 3 <= 1.6500000357627869)
     Predict: 1.0
    Else (feature 3 > 1.6500000357627869)
     If (feature 0 <= 5.950000047683716)
      Predict: 1.0
     Else (feature 0 > 5.950000047683716)
      Predict: 2.0
   Else (feature 2 > 4.8500001430511475)
    If (feature 3 <= 1.75)
     If (feature 2 <= 4.950000047683716)
      Predict: 1.0
     Else (feature 2 > 4.950000047683716)
      If (feature 3 <= 1.550000011920929)
       Predict: 2.0
      Else (feature 3 > 1.550000011920929)
       Predict: 1.0
    Else (feature 3 > 1.75)
     Predict: 2.0

PREDICTIONS
+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[5.0,3.4000000953...|       0.0|
|[5

# Decision Tree - MODEL TESTING

In [40]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_dtree = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy_dtree = evaluator_dtree.evaluate(predictions, 
                  {evaluator_dtree.metricName: "accuracy"})
print(f"Accuracy: {accuracy_dtree}")
precision_dtree = evaluator_dtree.evaluate(predictions,
                  {evaluator_dtree.metricName: "weightedPrecision"})
print(f"Precision: {precision_dtree}")
recall_dtree = evaluator_dtree.evaluate(predictions,
                  {evaluator_dtree.metricName: "weightedRecall"})
print(f"Recall: {recall_dtree}")
f1_dtree = evaluator_dtree.evaluate(predictions,
                {evaluator_dtree.metricName: "f1"})
print(f"F1 Score: {f1_dtree}")   

Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545


# SVM - Iris

In [41]:
from pyspark.ml.classification import LinearSVC, OneVsRest

# Initialize and configure the LinearSVC model ADD oneVsRest
lsvc = LinearSVC(maxIter=10, regParam=0.01) # regParam para el overfitting <= 10% o 0.1
ovr = OneVsRest(classifier=lsvc)

# Train SVM Model
iris_svm_model = ovr.fit(train_df)

# Predictions
predictions_svm = iris_svm_model.transform(test_df)


# Evaluate predictions SVM

In [43]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

accuracy_svm = evaluator.evaluate(predictions_svm, {evaluator.metricName: "accuracy"})
print(f"SVM Accuracy:  {accuracy_svm:.4f}")

precision_svm = evaluator.evaluate(predictions_svm, {evaluator.metricName: "weightedPrecision"})
print(f"SVM Precision: {precision_svm:.4f}")

recall_svm = evaluator.evaluate(predictions_svm, {evaluator.metricName: "weightedRecall"})
print(f"SVM Recall:    {recall_svm:.4f}")

f1_svm = evaluator.evaluate(predictions_svm, {evaluator.metricName: "f1"})
print(f"SVM F1 Score:  {f1_svm:.4f}")


SVM Accuracy:  0.8636
SVM Precision: 0.8662
SVM Recall:    0.8636
SVM F1 Score:  0.8627


# Comparisons F1 Score

In [44]:
print(f"F1 Score SVM: {f1_svm:.4f}")
print(f"F1 Score Decision Tree: {f1_dtree:.4f}")


F1 Score SVM: 0.8627
F1 Score Decision Tree: 0.9545


In [30]:
sc.stop()